### 모델 성능 평가
1. RAG + GPT로 답안지 만들기
2. M회사에서 fine-tunning한 모델과 지표 비교를 위한 각각의 답변 생성
3. 지표 비교

#### 1. RAG + GPT로 답안지 만들기

- RAG + GPT 테스트

In [ ]:
# GPU 성능을 높이기 위한 라이브러리
!pip install -U accelerate==0.29.3 peft==0.10.0 bitsandbytes==0.45.2
!pip install fsspec==2024.12.0 gcsfs==2024.12.0
!pip install --upgrade datasets
!pip install -U transformers==4.41.0 trl==0.8.6
!pip install faiss-cpu # 내가 만든 클래스에서 faiss 라이브러리를 사용하기 위해

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import sys

# 특정 디렉토리를 모듈 검색 경로에 추가
sys.path.append('/content/drive/MyDrive/healthcare project')

In [ ]:
import pandas as pd
import torch
import joblib
import pandas as pd
import warnings
import textwrap
from sklearn.exceptions import DataConversionWarning
from transformers import(
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig, # 메모리 사용량을 줄이기 위해 양자화
    # pipeline
)
from transformers import logging
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import LabelEncoder
from SearchAnswers import SelectReferenceAnswer # 직접 만든 클래스
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# 저장된 RandomForestClassifier 모델 불러오기
rf_model = joblib.load('/content/drive/MyDrive/healthcare project/headache_randomforest.pkl')
scaler = joblib.load('/content/drive/MyDrive/healthcare project/headache_scaler.pkl')
pca = joblib.load('/content/drive/MyDrive/healthcare project/headache_pca.pkl')

answer_df = pd.read_csv("/content/drive/MyDrive/healthcare project/두통 답변.csv", index_col=0)
embed_model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')
labelencoder = LabelEncoder()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# 객체 생성
select_answers = SelectReferenceAnswer(answer_df, scaler, pca, rf_model, embed_model, labelencoder)

In [ ]:
# 벡터 DB 만들기
intention_map, answer_map = select_answers.make_vec_db()

In [ ]:
question = '긴장하면 두통이 생길수도 있나요?' # '오른쪽 머리에서 찌릿하는 두통은 뭔가요?'
rag_result = select_answers.search_answer(question, intention_map, answer_map, 5)
rag_result

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[{'참고 정보1: 두통은 다양한 원인에 의해 발생하는 두통을 나타내는 용어입니다. 일반적인 두통은 일상 생활에 큰 영향을 미치지 않는 경우가 많습니다. 그러나 두통이 심해지거나 지속되면 생활의 질을 저하시키고 다른 증상도 동반될 수 있습니다. 예를 들어, 긴장성 두통은 일상적인 스트레스, 근육 통증과 피로, 신경 과민 등에 의해 악화될 수 있습니다. 이는 일상 생활에 지장을 줄 수 있습니다. 신경성 두통은 신경의 과도한 자극이 원인일 수 있습니다. 두통의 원인이 되는 질병의 예로는 뇌졸중, 뇌종양, 뇌염 등이 있습니다. 또한, 스트레스와 긴장이 결합하여 일시적으로 두통이 발생하기도 합니다. 두통을 효과적으로 관리하려면 원인을 파악하는 것이 중요합니다. 만약 두통이 지속된다면, 의료 전문가의 상담을 받는 것이 좋습니다. 의료 전문가의 조언을 받으면 두통 증상을 완화시키고 일상 생활의 질을 높일 수 있습니다.'},
 {'참고 정보2: 두통은 흔한 질환으로, 원인은 다양합니다. 편두통의 주요 증상은 통증의 성격, 지속 시간, 심각성 및 강도입니다. 일반적으로 두통은 통증이 한 쪽 눈에 집중되며, 주로 아침에 더 심해집니다. 편두통은 머리의 한쪽 부분에 극심한 통증을 유발하며, 두통이 발생한 후에는 30분에서 1시간 후에 두통이 완화될 수 있습니다. 편두통은 식사 도중이나 운동 도중에 통증이 나타나기도 합니다. 또한, 두통이 갑자기 발생하며, 머리를 꽉 누르거나 압박감이 있을 수 있습니다. 두통은 매우 강렬하며, 두통이 사라진 후에도 몇 시간 동안 지속될 수 있습니다. 특히, 두통이 한 달 이상 지속되거나, 일상 생활에 큰 영향을 미칠 정도로 심한 경우 병원을 방문하여 진찰을 받아야 합니다. 편두통은 일상생활에 큰 지장을 줄 수 있는 만성적인 질환이며, 증상의 경중에 따라 심각성이 달라집니다. 일상 생활에 지장이 있는 경우 의료 전문가의 도움을 받는 것이 중요합니다.'},
 {'참고 정보3: 두통은 신체의 여러 기관에 손상이 없어도 발생할 수 있습니다. 이는 두통을

In [ ]:
def make_ragresult_context(rag_result):
    rag_result_texts = [list(i)[0] for i in rag_result]
    rag_result_context = '\n\n'.join(rag_result_texts)
    return rag_result_context

rag_context = make_ragresult_context(rag_result)
print(rag_context)

참고 정보1: 두통은 다양한 원인에 의해 발생하는 두통을 나타내는 용어입니다. 일반적인 두통은 일상 생활에 큰 영향을 미치지 않는 경우가 많습니다. 그러나 두통이 심해지거나 지속되면 생활의 질을 저하시키고 다른 증상도 동반될 수 있습니다. 예를 들어, 긴장성 두통은 일상적인 스트레스, 근육 통증과 피로, 신경 과민 등에 의해 악화될 수 있습니다. 이는 일상 생활에 지장을 줄 수 있습니다. 신경성 두통은 신경의 과도한 자극이 원인일 수 있습니다. 두통의 원인이 되는 질병의 예로는 뇌졸중, 뇌종양, 뇌염 등이 있습니다. 또한, 스트레스와 긴장이 결합하여 일시적으로 두통이 발생하기도 합니다. 두통을 효과적으로 관리하려면 원인을 파악하는 것이 중요합니다. 만약 두통이 지속된다면, 의료 전문가의 상담을 받는 것이 좋습니다. 의료 전문가의 조언을 받으면 두통 증상을 완화시키고 일상 생활의 질을 높일 수 있습니다.

참고 정보2: 두통은 흔한 질환으로, 원인은 다양합니다. 편두통의 주요 증상은 통증의 성격, 지속 시간, 심각성 및 강도입니다. 일반적으로 두통은 통증이 한 쪽 눈에 집중되며, 주로 아침에 더 심해집니다. 편두통은 머리의 한쪽 부분에 극심한 통증을 유발하며, 두통이 발생한 후에는 30분에서 1시간 후에 두통이 완화될 수 있습니다. 편두통은 식사 도중이나 운동 도중에 통증이 나타나기도 합니다. 또한, 두통이 갑자기 발생하며, 머리를 꽉 누르거나 압박감이 있을 수 있습니다. 두통은 매우 강렬하며, 두통이 사라진 후에도 몇 시간 동안 지속될 수 있습니다. 특히, 두통이 한 달 이상 지속되거나, 일상 생활에 큰 영향을 미칠 정도로 심한 경우 병원을 방문하여 진찰을 받아야 합니다. 편두통은 일상생활에 큰 지장을 줄 수 있는 만성적인 질환이며, 증상의 경중에 따라 심각성이 달라집니다. 일상 생활에 지장이 있는 경우 의료 전문가의 도움을 받는 것이 중요합니다.

참고 정보3: 두통은 신체의 여러 기관에 손상이 없어도 발생할 수 있습니다. 이는 두통을 유발하는 다양한 요인에

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "API키"

In [ ]:
from openai import OpenAI

def gpt_result(user_input, rag_context):

    client = OpenAI()

    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[

            # 시스템 명령어(없어도됨)
            {"role": "system", "content": "너는 의료 전문가 챗봇이야. 질문에 최대한 정확하고 자연스러운 대화체로 답변해줘. 해당 참고 정보를 참고해서 질문에 답변해줘."},

            # 본격적인 명령어
            {
                "role": "user",
                "content": f"""
                [참고 정보]
                {rag_context}

                [질문]
                {user_input}""" # 내가 입력하고 싶은 명령어
            }
        ]
    )

    return completion.choices[0].message.content

In [ ]:
user_input = '긴장하면 두통이 생길수도 있나요?'
response = gpt_result(user_input, rag_context)

In [ ]:
response

'네, 긴장은 두통의 원인 중 하나일 수 있습니다. 긴장성 두통은 일상적인 스트레스, 근육 통증과 피로, 신경 과민 등에 의해 악화될 수 있습니다. 긴장성 두통은 주로 일상 생활에서의 스트레스나 긴장으로 인해 발생하며, 두통이 가끔씩 발생하거나 지속적으로 발생할 수 있습니다. 따라서, 긴장 상태가 지속되면 두통이 발생할 수 있으며, 이는 일상 생활에 영향을 줄 정도로 심해질 수도 있습니다. 만약 두통이 지속되거나 심해진다면 의료 전문가의 상담을 받는 것이 좋습니다. 의료 전문가의 조언을 받으면 두통 증상을 완화시키고 일상 생활의 질을 높일 수 있습니다.'

- 답안지 만들기

In [ ]:
# 데이터셋 로드
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/healthcare project/headache_qa_matching.csv', index_col=0)
df.head(2)

,disease_category,disease_name,intention,question,final_answer
0,응급질환,두통,예방,두통 예방을 위해 어떤 자세나 활동이 도움이 될까요?,만약 일상적인 예방 및 조절 조치를 취하는 것이 어렵거나 효과가 없다면 다음과 같은...
1,응급질환,두통,예방,스트레스를 많이 받는 사람이 두통을 예방하기 위해 어떤 생활습관을 가져야 할까요?,두통은 많은 사람들이 일상적으로 겪는 현상입니다. 일반적인 예방 방법으로는 규칙적인...


In [ ]:
# 평가용으로 사용할 질문 데이터셋
val_question_df = pd.read_csv('/content/drive/MyDrive/healthcare project/val_question.csv', index_col=0)
val_question_df

,question
0,"식이나 생활 습관이 두통, 코막힘, 구토, 설사와 같은 증상을 유발하는 이유에 대해..."
1,두통이라는 질병의 정확한 정의를 알려주세요.
2,"두통, 어지러움, 구토 등의 호흡기 및 중추신경계 증상이 발생하는데 영향을 주는 주..."
3,두통의 병적인 진단을 위해 어떤 기준이 고려되는지 알려주세요.
4,일산화탄소 중독의 진단 기준에 대해 자세히 알려주세요.
...,...
98,두통이 심한데 스트레스성 말고 다른 원인들이 무엇이 있는지 알려주세요.
99,호흡성 산증을 진단하기 위해 어떤 증상을 주의깊게 관찰해야 할까요?
100,"두통이 점점 심해지는 경우, 정확한 진단을 받기 위해서는 어떤 의료 기관을 찾아야 ..."
101,두통 치료를 위해 병원에서 어떤 치료 방법을 시행하는지 알려주세요.


In [ ]:
gpt_result_list = []

for question in val_question_df['question']:
    rag_result = select_answers.search_answer(question, intention_map, answer_map, 5)
    rag_context = make_ragresult_context(rag_result)

    gpt_result_list.append({'질문': question, '답변': gpt_result(question, rag_context)})

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,

In [ ]:
len(gpt_result_list), gpt_result_list[0]

(103,
 {'질문': '식이나 생활 습관이 두통, 코막힘, 구토, 설사와 같은 증상을 유발하는 이유에 대해 자세한 설명을 듣고 싶습니다.',
  '답변': '식이나 생활 습관이 두통, 코막힘, 구토, 설사와 같은 증상을 유발하는 이유에 대해 자세한 설명해드리겠습니다. 일반적으로 식이나 생활 습관이 두통 및 다른 증상을 유발하는 주요 이유는 다음과 같습니다.\n\n1. 식이 요인: \n   - **음식 알레르기**: 어떤 음식물에 알레르기 반응이 생겨 두통을 유발할 수 있습니다.\n   - **음식 성분**: 식이 성분에 따라 두통을 유발할 수 있습니다. 예를 들어, 카페인이 들어간 음료나 초콜릿은 많은 사람에게 두통을 유발할 수 있습니다.\n   - **짜거나 맵거나 자극적인 음식**: 너무 짠 음식이나 매운 음식은 혈액순환을 증가시키거나 소화를 방해하여 두통을 유발할 수 있습니다.\n\n2. 생활 습관: \n   - **스트레스**: 스트레스는 두통의 주요 원인 중 하나로, 긴장성 두통을 일으킬 수 있습니다.\n   - **수면 부족**: 충분한 휴식을 취하지 않거나 수면 부족이 생길 경우, 두통을 유발할 수 있습니다.\n   - **약물 남용**: 과도한 알코올 섭취나 흡연과 같은 약물 남용은 두통을 유발할 수 있습니다.\n   - **음주**: 과도한 음주는 신경계통 불균형을 일으켜 두통을 유발할 수 있습니다.\n\n따라서, 개인의 식이 및 생활 습관을 조절하고, 스트레스를 관리하며 충분한 휴식을 취하는 것이 두통 및 관련 증상을 예방하고 완화하는 데 도움이 될 수 있습니다. 하지만, 만약 두통이 지속되거나 심각한 증상을 동반할 경우에는 반드시 의사와 상담하여 원인을 파악하고 적절한 치료를 받는 것이 중요합니다.'})

In [ ]:
gpt_result_df = pd.DataFrame(gpt_result_list)
gpt_result_df.to_csv('/content/drive/MyDrive/healthcare project/gpt_result.csv')

In [ ]:
pd.read_csv('/content/drive/MyDrive/healthcare project/gpt_result.csv', index_col=0)

,질문,답변
0,"식이나 생활 습관이 두통, 코막힘, 구토, 설사와 같은 증상을 유발하는 이유에 대해...","식이나 생활 습관이 두통, 코막힘, 구토, 설사와 같은 증상을 유발하는 이유에 대해..."
1,두통이라는 질병의 정확한 정의를 알려주세요.,"두통은 일상생활에서 흔히 경험하는 증상 중 하나로, 머리의 어느 부분에서든 발생할 ..."
2,"두통, 어지러움, 구토 등의 호흡기 및 중추신경계 증상이 발생하는데 영향을 주는 주...","주요한 원인으로는 긴장성 두통과 편두통이 있습니다. 긴장성 두통은 긴장, 불안, 우..."
3,두통의 병적인 진단을 위해 어떤 기준이 고려되는지 알려주세요.,두통의 병적인 진단을 위해 고려되는 기준은 다음과 같습니다.\n\n1. 병력 확인:...
4,일산화탄소 중독의 진단 기준에 대해 자세히 알려주세요.,일산화탄소 중독의 진단은 주로 환자의 증상과 의사의 진료를 통해 이루어집니다. 하지...
...,...,...
98,두통이 심한데 스트레스성 말고 다른 원인들이 무엇이 있는지 알려주세요.,"두통은 다양한 원인에 의해 발생할 수 있습니다. 스트레스 외에도 뇌 내의 문제, 혈..."
99,호흡성 산증을 진단하기 위해 어떤 증상을 주의깊게 관찰해야 할까요?,호흡성 산증을 진단하기 위해 주의깊게 관찰해야 할 증상은 다음과 같습니다:\n1. ...
100,"두통이 점점 심해지는 경우, 정확한 진단을 받기 위해서는 어떤 의료 기관을 찾아야 ...",두통이 점점 심해진다면 정확한 진단을 받기 위해서는 먼저 일반적인 병원이 아닌 신경...
101,두통 치료를 위해 병원에서 어떤 치료 방법을 시행하는지 알려주세요.,두통 치료를 위해 병원에서 다양한 치료 방법을 시행할 수 있습니다. 대표적으로 약물...


#### 2. 지표 비교를 위한 각각의 답변 생성

- 내가 학습시킨 모델 답변 생성

In [ ]:
# 내부 경고출력x
warnings.filterwarnings("ignore", category=DataConversionWarning)
logging.set_verbosity_error()

# 학습된 모델 사용을 위한 기본 설정
# 학습된 모델 및 토크나이저 불러오기
model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/full_trained_model_instruct", device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/full_trained_model_instruct")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# 답변 생성 함수
def chat_response(model, tokenizer, user_input):

    rag_result = select_answers.search_answer(user_input, intention_map, answer_map, 3)
    rag_context = make_ragresult_context(rag_result)

    messages = [
        {"role": "system", "content": "너는 의료 전문가 챗봇이야. 질문에 최대한 정확하고 자연스러운 대화체로 답변해줘. 해당 참고 정보를 참고해서 질문에 답변해줘."},
        {"role": "user", "content": f"""

        [참고 정보]
        {rag_context}

        [질문]
        {user_input}"""}
    ]

    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    # 생성 종료 조건을 정해줌.
    terminators = [
        tokenizer.eos_token_id, # 이 토큰을 만나면 생성 종료
        tokenizer.convert_tokens_to_ids("<|eot_id|>") # <|eot_id|> 이 토큰 만나도 생성 종료
    ]

    outputs = model.generate(
        input_ids,
        attention_mask=(input_ids != tokenizer.pad_token_id), # 패딩이 아닌 실제 텍스트만 골라서 마스크 생성(마스크: 필요한 정보만 놔두고 나머지 가림)
        max_new_tokens=512,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.7,
        top_p=0.8,
    )
    response = outputs[0][input_ids.shape[-1]:]
    result = tokenizer.decode(response, skip_special_tokens=True)

    def print_wrapped_text(text, width=80):
        wrapped_text = textwrap.fill(text, width=width)
        print(wrapped_text)

    return textwrap.fill(result, width=80)

In [ ]:
my_result_list = []

for question in val_question_df['question']:

    my_result_list.append({'질문': question, '답변': chat_response(model, tokenizer, question)})

In [ ]:
len(my_result_list)

103

In [ ]:
my_model_df = pd.DataFrame(my_result_list)
my_model_df.to_csv('/content/drive/MyDrive/healthcare project/my_model_result.csv')

In [ ]:
import pandas as pd
my_model_df = pd.read_csv('/content/drive/MyDrive/healthcare project/my_model_result.csv', index_col=0)
my_model_df.head(2)

,질문,답변
0,"식이나 생활 습관이 두통, 코막힘, 구토, 설사와 같은 증상을 유발하는 이유에 대해...","두통은 다양한 원인에 의해 발생할 수 있습니다. 스트레스, 피로, 수면 부족, 약물..."
1,두통이라는 질병의 정확한 정의를 알려주세요.,두통은 일차성 두통과 이차성 두통으로 구분됩니다. 일차성 두통은 다양한 원인으로 발...


#### 3. 지표 비교

In [1]:
!pip install evaluate
!pip install bert_score
!pip install rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 59.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
 

In [2]:
import pandas as pd

gpt_result_df = pd.read_csv('/content/drive/MyDrive/healthcare project/gpt_result.csv', index_col=0) # 답안지
my_model_df = pd.read_csv('/content/drive/MyDrive/healthcare project/my_model_result.csv', index_col=0) # 내 모델
company_result_df = pd.read_csv('/content/drive/MyDrive/healthcare project/M_Company_model_result.csv', index_col=0) # 데이터 제공 업체 모델

In [3]:
from evaluate import load

bertscore = load("bertscore")
bleu = load("bleu")
rouge = load("rouge")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
gpt_answer_list = [a for a in gpt_result_df['답변']]
my_answer_list = [a for a in my_model_df['답변']]
company_answer_list = [a for a in company_result_df['answer']]

In [5]:
# BERT Score 문장의 의미적 유사도
bert_result_my_model = bertscore.compute(predictions=my_answer_list, references=gpt_answer_list, lang="ko")
bert_result_company = bertscore.compute(predictions=company_answer_list, references=gpt_answer_list, lang="ko")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

In [6]:
# BLEU:내가 생성한 결과가 레퍼런스(정답)에 얼마나 등장하는지(Precision 관점)
bleu_result_my_model = bleu.compute(predictions=my_answer_list, references=[[r] for r in gpt_answer_list])
bleu_result_company = bleu.compute(predictions=company_answer_list, references=[[r] for r in gpt_answer_list])

# ROUGE:레퍼런스(정답)가 내가 생성한 결과에 얼마나 포함되는지(Recall 관점)
rouge_result_my_model = rouge.compute(predictions=my_answer_list, references=gpt_answer_list)
rouge_result_company = rouge.compute(predictions=company_answer_list, references=gpt_answer_list)

In [7]:
bleu_result_my_model['bleu'], bleu_result_company['bleu']

(0.10719189689792218, 0.0592628381970711)

In [8]:
 rouge_result_my_model['rougeL'], rouge_result_company['rougeL']

(np.float64(0.021910856134157105), np.float64(0.016828478964401296))

In [9]:
import numpy as np
bert_score_my = np.mean([fs for fs in bert_result_my_model['f1']])
bert_score_company = np.mean([fs for fs in bert_result_company['f1']])
bert_score_my, bert_score_company

(np.float64(0.753793374427314), np.float64(0.7437584116620924))

In [10]:
pd.DataFrame({
    'Bert Score': [bert_score_my, bert_score_company],
    'Bleu': [bleu_result_my_model['bleu'], bleu_result_company['bleu']],
    'RougeL': [rouge_result_my_model['rougeL'], rouge_result_company['rougeL']]
})

,Bert Score,Bleu,RougeL
0,0.753793,0.107192,0.021911
1,0.743758,0.059263,0.016828


In [11]:
pd.DataFrame({
    'company_model': [ bert_score_company, bleu_result_company['bleu'], rouge_result_company['rougeL']],
    'my_model': [bert_score_my, bleu_result_my_model['bleu'], rouge_result_my_model['rougeL']],
}, index=['Bert Score', 'Bleu', 'RougeL'])

,company_model,my_model
Bert Score,0.743758,0.753793
Bleu,0.059263,0.107192
RougeL,0.016828,0.021911
